# Connecting to MongoDB using pymongo

Maciej Kondrat
<br>
tarpalsus@gmail.com
<br>
PyData Wroclaw - 13/09/2017



## Short plan:
    
- Mongo and NoSQL overview
- Setup and database connection
- Inserting and deleting records
- Querying
- MapReduce support and simple use case

# What is a NoSQL database?

## Basic types:

- Document based (MongoDB)
![mongo.png](mongo.png)
- Key- value based (Redis)
![redis.png](redis.png)
- Column based (Cassandra)
![cassandra.png](cassandra.png)
- Graph based (Neo4j)
![neo.png](neo.png)
    

## MongoDB:

- Since 2007
- Files stored as BSON
- Indexing
- Load balancing
- MapReduce
- JavaScript execution

## Running mongo on Windows

## Using pymongo

In [1]:
import pymongo
from pymongo import MongoClient
#Default is localhost
client = MongoClient()
#Creation of database couldn't get any easier
db = client.test

## Inserting data

In [2]:
db.restaurants.delete_many({})
from datetime import datetime
result = db.restaurants.insert_one(
    {
        "address": {
            "street": "2 Avenue",
            "zipcode": "10075",
            "building": "100",
            "coord": [-73.9557413, 40.7720266]
        },
        "borough": "Manhattan",
        "cuisine": "Italian",
        "grade": "A",
        "score": 11,

        "name": "Vella",
        "restaurant_id": "41704620"
    }
)
result = db.restaurants.insert_one(
    {
        "address": {
            "street": "5 Avenue",
            "zipcode": "10075",
            "building": "1480",
            "coord": [-72.9557413, 40.7520266]
        },
        "borough": "Manhattan",
        "cuisine": "Grill",
        "grade": "C",
        "score": 5,
        "name": "Jerry's",
        "restaurant_id": "41704621"
    }
)

In [3]:
json_data = [{
        "address": {
            "street": "8 Avenue",
            "zipcode": "10075",
            "building": "1480",
            "coord": [-73.9557413, 40.7720266]
        },
        "borough": "Wroclaw",
        "cuisine": "Italian",
        "grade": "B",
        "score": 8,
        "name": "Vellano",
        "restaurant_id": "3"
    },{
        "address": {
            "street": "Szewska",
            "zipcode": "-",
            "building": "40",
            "coord": [-73.9557413, 40.7720266]
        },
        "borough": "Wroclaw",
        "cuisine": "Italian",
        "grade": "A",
        "score": 15,
        "name": "Piec",
        "restaurant_id": "20"
    }]
result = db.restaurants.insert_many(json_data)

## Querying

In [4]:
print('Simple query\n')
cursor = db.restaurants.find({"name":'Vellano'})

for document in cursor:
    print(document)
cursor.close()

Simple query

{'cuisine': 'Italian', 'restaurant_id': '3', '_id': ObjectId('59b6c1422b66db3af048a31c'), 'borough': 'Wroclaw', 'name': 'Vellano', 'grade': 'B', 'address': {'zipcode': '10075', 'street': '8 Avenue', 'coord': [-73.9557413, 40.7720266], 'building': '1480'}, 'score': 8}


In [5]:
print('Score query\n')
cursor = db.restaurants.find({"score": {"$gt": 10}})
for document in cursor:
    print(document)
cursor.close()
print('\nGrade query\n')
cursor = db.restaurants.find({"grade": {"$in": ['C','B']}})
for document in cursor:
    print(document)
cursor.close()

Score query

{'cuisine': 'Italian', 'restaurant_id': '41704620', '_id': ObjectId('59b6c13f2b66db3af048a31a'), 'borough': 'Manhattan', 'name': 'Vella', 'grade': 'A', 'address': {'zipcode': '10075', 'street': '2 Avenue', 'coord': [-73.9557413, 40.7720266], 'building': '100'}, 'score': 11}
{'cuisine': 'Italian', 'restaurant_id': '20', '_id': ObjectId('59b6c1422b66db3af048a31d'), 'borough': 'Wroclaw', 'name': 'Piec', 'grade': 'A', 'address': {'zipcode': '-', 'street': 'Szewska', 'coord': [-73.9557413, 40.7720266], 'building': '40'}, 'score': 15}

Grade query

{'cuisine': 'Grill', 'restaurant_id': '41704621', '_id': ObjectId('59b6c13f2b66db3af048a31b'), 'borough': 'Manhattan', 'name': "Jerry's", 'grade': 'C', 'address': {'zipcode': '10075', 'street': '5 Avenue', 'coord': [-72.9557413, 40.7520266], 'building': '1480'}, 'score': 5}
{'cuisine': 'Italian', 'restaurant_id': '3', '_id': ObjectId('59b6c1422b66db3af048a31c'), 'borough': 'Wroclaw', 'name': 'Vellano', 'grade': 'B', 'address': {'zipco

## Deleting

In [6]:
result = db.restaurants.delete_many({})

cursor = db.restaurants.find({"name":'Vellano'})
print('Simple query\n')
for document in cursor:
    print(document)
cursor.close()

Simple query



## Simple book database

In [7]:
from bson.code import Code
client = MongoClient()
db = client.books_db
table = db.books
def map_reduce_for_words(table, author):
    map_query = Code("function () {"
           "  this.Text[0].forEach(function(z) {"
          "    emit(z, 1);"
         "  });"
          "}")
    reduce_query = Code("function (key, values) {"
              "  var total = 0;"
             "  for (var i = 0; i < values.length; i++) {"
             "    total += values[i];"
              "  }"
             "  return total;"
              "}")

    results = table.map_reduce(map_query,
                               reduce_query, "Word results",
                               query={"Book": author} )
    #for result in results.find():
     #   print(result['_id'], result['value'])
    return results

results = map_reduce_for_words(table, 'dosto')
sorted_result = sorted([(x['_id'],x['value']) for x in results.find()], key= lambda tup: tup[1], reverse=True)
print(*sorted_result, sep= '\n')

('the', 26168.0)
('and', 18975.0)
('to', 16442.0)
('of', 13143.0)
('he', 12609.0)
('you', 11351.0)
('that', 10219.0)
('in', 9247.0)
('it', 9221.0)
('was', 7910.0)
('his', 6839.0)
('for', 5510.0)
('but', 5307.0)
('at', 5248.0)
('not', 5207.0)
('with', 5047.0)
('him', 4963.0)
('had', 4956.0)
('is', 4475.0)
('as', 4005.0)
('all', 3991.0)
('me', 3845.0)
('she', 3697.0)
('her', 3689.0)
('have', 3574.0)
('on', 3556.0)
('be', 3210.0)
('so', 2956.0)
('this', 2912.0)
('my', 2858.0)
('what', 2838.0)
('one', 2375.0)
('there', 2244.0)
('from', 2238.0)
('are', 2223.0)
('said', 2221.0)
('they', 2151.0)
('no', 2084.0)
('if', 1966.0)
('by', 1955.0)
('prince', 1924.0)
('your', 1912.0)
('would', 1830.0)
('were', 1757.0)
('now', 1683.0)
('been', 1675.0)
('do', 1667.0)
('up', 1655.0)
('will', 1652.0)
('out', 1650.0)
('an', 1641.0)
('know', 1602.0)
('about', 1595.0)
('only', 1517.0)
('them', 1498.0)
('or', 1474.0)
('who', 1472.0)
('did', 1466.0)
('man', 1402.0)
('very', 1393.0)
('we', 1375.0)
('when', 1346

('handed', 32.0)
('hasn', 32.0)
('hastened', 32.0)
('herzenstube', 32.0)
('iii', 32.0)
('image', 32.0)
('invalid', 32.0)
('kindness', 32.0)
('knight', 32.0)
('lately', 32.0)
('lower', 32.0)
('mankind', 32.0)
('merry', 32.0)
('obliged', 32.0)
('plain', 32.0)
('playing', 32.0)
('proof', 32.0)
('punishment', 32.0)
('quarrel', 32.0)
('sad', 32.0)
('shed', 32.0)
('simplicity', 32.0)
('unconscious', 32.0)
('audience', 31.0)
('believing', 31.0)
('bielokonski', 31.0)
('bitterly', 31.0)
('brandy', 31.0)
('clerk', 31.0)
('darling', 31.0)
('driven', 31.0)
('eat', 31.0)
('endure', 31.0)
('exhausted', 31.0)
('fellows', 31.0)
('gentle', 31.0)
('group', 31.0)
('hall', 31.0)
('ii', 31.0)
('individual', 31.0)
('investigating', 31.0)
('invited', 31.0)
('irritation', 31.0)
('material', 31.0)
('melancholy', 31.0)
('mere', 31.0)
('occurred', 31.0)
('original', 31.0)
('pardon', 31.0)
('plainly', 31.0)
('pressed', 31.0)
('recognize', 31.0)
('reflected', 31.0)
('respectable', 31.0)
('style', 31.0)
('uneasy', 

('requirements', 8.0)
('resolute', 8.0)
('respects', 8.0)
('responsibility', 8.0)
('retreat', 8.0)
('returns', 8.0)
('reverently', 8.0)
('rightful', 8.0)
('rigmarole', 8.0)
('rivalry', 8.0)
('roo', 8.0)
('rubbing', 8.0)
('rubbish', 8.0)
('ruining', 8.0)
('rumours', 8.0)
('runs', 8.0)
('sacrilege', 8.0)
('salaskin', 8.0)
('sayings', 8.0)
('searching', 8.0)
('seeks', 8.0)
('sentimental', 8.0)
('severity', 8.0)
('sex', 8.0)
('sixth', 8.0)
('snatch', 8.0)
('sole', 8.0)
('sorrowfully', 8.0)
('speeches', 8.0)
('speed', 8.0)
('spoil', 8.0)
('stamping', 8.0)
('status', 8.0)
('steadfast', 8.0)
('stirring', 8.0)
('stove', 8.0)
('stretching', 8.0)
('strictly', 8.0)
('strode', 8.0)
('stuck', 8.0)
('subsided', 8.0)
('subtle', 8.0)
('sunday', 8.0)
('supper', 8.0)
('tearful', 8.0)
('tempted', 8.0)
('theatre', 8.0)
('theirs', 8.0)
('thirsting', 8.0)
('thorough', 8.0)
('thunder', 8.0)
('thursday', 8.0)
('ticket', 8.0)
('timofeyovitch', 8.0)
('tirade', 8.0)
('tortures', 8.0)
('torturing', 8.0)
('tower',

('seals', 4.0)
('seduced', 4.0)
('select', 4.0)
('selling', 4.0)
('semblance', 4.0)
('semi', 4.0)
('sensualists', 4.0)
('sequence', 4.0)
('serf', 4.0)
('series', 4.0)
('servile', 4.0)
('settlement', 4.0)
('seville', 4.0)
('shalt', 4.0)
('sharing', 4.0)
('shaven', 4.0)
('shedding', 4.0)
('sheepish', 4.0)
('shelter', 4.0)
('shepherd', 4.0)
('shivered', 4.0)
('shrieks', 4.0)
('sighted', 4.0)
('simpler', 4.0)
('sinister', 4.0)
('sinks', 4.0)
('sipping', 4.0)
('sixteen', 4.0)
('slammed', 4.0)
('slandered', 4.0)
('sleeper', 4.0)
('smouldering', 4.0)
('snap', 4.0)
('softness', 4.0)
('sohn_', 4.0)
('soundly', 4.0)
('sources', 4.0)
('spark', 4.0)
('spasm', 4.0)
('spitefully', 4.0)
('split', 4.0)
('spoiled', 4.0)
('spoiling', 4.0)
('spree', 4.0)
('stab', 4.0)
('stammering', 4.0)
('stamp', 4.0)
('starvation', 4.0)
('statements', 4.0)
('stationed', 4.0)
('steel', 4.0)
('stepanida', 4.0)
('sticky', 4.0)
('stifle', 4.0)
('sting', 4.0)
('stings', 4.0)
('stocking', 4.0)
('stomach', 4.0)
('stoutly', 4.

('molecule', 2.0)
('molehill', 2.0)
('momentarily', 2.0)
('monsieur', 2.0)
('moody', 2.0)
('morals', 2.0)
('morbidly', 2.0)
('mort', 2.0)
('mortals', 2.0)
('mortgaged', 2.0)
('mortifying', 2.0)
('mostly', 2.0)
('motioning', 2.0)
('motto', 2.0)
('mountebank', 2.0)
('mouthful', 2.0)
('mouths', 2.0)
('muddled', 2.0)
('mug', 2.0)
('multiplication', 2.0)
('murderous', 2.0)
('murmur', 2.0)
('muscular', 2.0)
('muse', 2.0)
('musingly', 2.0)
('mustaches', 2.0)
('muster', 2.0)
('mutton', 2.0)
('nag', 2.0)
('nails', 2.0)
('naiveness', 2.0)
('naivete', 2.0)
('narrowed', 2.0)
('nastasya', 2.0)
('nastia', 2.0)
('nastiest', 2.0)
('natasha', 2.0)
('nation', 2.0)
('nationality', 2.0)
('naïvely', 2.0)
('necessarily', 2.0)
('needless', 2.0)
('nekrassov', 2.0)
('nelyudov', 2.0)
('net', 2.0)
('nevsky', 2.0)
('newby', 2.0)
('niece', 2.0)
('nieces', 2.0)
('nihil', 2.0)
('nihilism', 2.0)
('nikitishna', 2.0)
('nineteen', 2.0)
('nobles', 2.0)
('noised', 2.0)
('nominal', 2.0)
('nonproprietary', 2.0)
('noteworthy

('duller', 1.0)
('dullest', 1.0)
('duly', 1.0)
('dumas', 1.0)
('dun', 1.0)
('dupe', 1.0)
('duplicity', 1.0)
('dusty', 1.0)
('dwarf', 1.0)
('dwarfish', 1.0)
('dweller', 1.0)
('dwellings', 1.0)
('dwindled', 1.0)
('dynasty', 1.0)
('dyspeptic', 1.0)
('décharge_', 1.0)
('déluge_', 1.0)
('eagles', 1.0)
('earthquake', 1.0)
('earthy', 1.0)
('eased', 1.0)
('eats', 1.0)
('eavesdropping', 1.0)
('ebb', 1.0)
('eccentricities', 1.0)
('echoing', 1.0)
('economic', 1.0)
('economical', 1.0)
('economically', 1.0)
('economist', 1.0)
('economized', 1.0)
('economizing', 1.0)
('edict', 1.0)
('editors', 1.0)
('educating', 1.0)
('edwards', 1.0)
('effacement', 1.0)
('effected', 1.0)
('effectual', 1.0)
('efficient', 1.0)
('ego', 1.0)
('egoistic', 1.0)
('egoists', 1.0)
('egotist', 1.0)
('egotistical', 1.0)
('egyptian', 1.0)
('eighth', 1.0)
('eisenschmidt', 1.0)
('ejaculate', 1.0)
('ejected', 1.0)
('ekaterinenburg', 1.0)
('ekshaisk', 1.0)
('ekshaisky', 1.0)
('elaborate', 1.0)
('elaborated', 1.0)
('elaborately', 1.

('smarten', 1.0)
('smartest', 1.0)
('smarting', 1.0)
('smashes', 1.0)
('smattering', 1.0)
('smelling', 1.0)
('smelt', 1.0)
('smilingly', 1.0)
('smirked', 1.0)
('smite', 1.0)
('smoked', 1.0)
('smoldered', 1.0)
('smolensk', 1.0)
('smoothing', 1.0)
('smoothly', 1.0)
('smote', 1.0)
('snake', 1.0)
('snakes', 1.0)
('snares', 1.0)
('snarling', 1.0)
('sne', 1.0)
('sneak', 1.0)
('sneers', 1.0)
('sneeze', 1.0)
('snegiryovs', 1.0)
('sniffed', 1.0)
('sniffing', 1.0)
('sniggering', 1.0)
('snored', 1.0)
('snowball', 1.0)
('snowy', 1.0)
('snub', 1.0)
('snubbing', 1.0)
('snubs', 1.0)
('soap', 1.0)
('soaped', 1.0)
('soaping', 1.0)
('soar', 1.0)
('sobakevitch', 1.0)
('socialistic', 1.0)
('socially', 1.0)
('sockets', 1.0)
('softens', 1.0)
('soit', 1.0)
('solace', 1.0)
('solar', 1.0)
('soldierly', 1.0)
('solicitor', 1.0)
('solidly', 1.0)
('somber', 1.0)
('sonorous', 1.0)
('sophia', 1.0)
('sorceress', 1.0)
('sorely', 1.0)
('sorrowing', 1.0)
('sorrows', 1.0)
('sot', 1.0)
('sotski', 1.0)
('sottish', 1.0)
('s

## Quick wrap up:
- pymongo is really easy to pick up
- It can help you with managing unstructured data
- js can be painful...
- Remember that in many cases it's better to use SQL or even .csv